# A Process For A Stock Prics

## Geometric Brownian Motion

A first guess at an appropriate process for model for a stock price could be that of generalised Brownian motion given by

$$dS = \mu dt + \sigma dz$$

this model implies that a change in the stock price will be given by

$$\Delta S = S_{t + \Delta t} - S_t = \mu \Delta t + \sigma \epsilon \sqrt{\Delta t}$$

the expected increase is given by $\mu \Delta t$. **This seems wrong, the growth in a stock price should be related to the size of the stock price**. We therfore model the percentage changes as following generalised Brownian motion.

$$ \frac{\Delta S}{S} = \frac{S_{t + \Delta t} - S_t}{S_t} = \mu \Delta t + \sigma \Delta z$$

rearanging and we arive at the equation for the Geometric Brownian motion of a stock price.

$$dS = \mu S dt + \sigma S dz$$

## The Lognormal Property

We may use Itô's lemma to establish how the logarithm of price changes behvaes. Recall that Itô's lemma is given by

$$\partial G = \left(\mu S \frac{\partial G}{\partial S} + \frac{\partial G}{\partial t} + \frac{1}{2} \sigma^2 S^2\frac{\partial^2 G}{\partial S^2} \right) dt + \sigma S \frac{\partial G}{\partial s}dz $$

If we consider a the transformation $$G = \ln S$$

then we have the following 

$$ \frac{\partial G}{\partial S} = \frac{1}{S} \text{ , }  \frac{\partial G}{\partial t} = 0 \text{ , } \frac{\partial^2 G}{\partial S^2} = -\frac{1}{S^2}$$

resulting in the following differential equation 

$$ \partial ln S = \left( \mu - \frac{\sigma^2}{2} \right)dt + \sigma dz $$

thus the logarithm of a stock price follows a generalised Wiener process with 

$$\text{drift} = \mu - \frac{\sigma^2}{2} \text{ , } \text{var rate} = \sigma^2$$

meaning that changes in price are distributed acording to

$$ ln\frac{S_{T}}{S_0} \sim \mathcal{N}\left[ \left(\mu - \frac{\sigma^2}{2} \right)T,  \sigma^2 T\right] $$

## Estimating $\mu$ and $\sigma$

In [38]:
import numpy as n
from statistics import variance

In [5]:
prices = np.array([136.05, 134.34, 134.2, 133.55, 134.9, 139.5, 137.5, 136.4, 136.697, 137.143, 137.838])

In [12]:
df = pd.DataFrame()

In [13]:
df['prices'] = prices

In [21]:
df['ratio'] = df['prices'][0:]/df['prices'].shift(1)

In [24]:
df['log_ratio'] = np.log(df['ratio'])

In [143]:
df

,prices,ratio,log_ratio
0,136.050,NaN,NaN
1,134.340,0.987431,-0.012649
2,134.200,0.998958,-0.001043
3,133.550,0.995156,-0.004855
4,134.900,1.010109,0.010058
5,139.500,1.034099,0.033531
6,137.500,0.985663,-0.014441
7,136.400,0.992000,-0.008032
8,136.697,1.002177,0.002175
9,137.143,1.003263,0.003257


In [217]:
x̄ = np.mean(df['log_ratio'][1:])
x̄

0.0013056617308833396

In [218]:
var = variance(df['log_ratio'][1:])

### Yearly variance 

We have been given daily data. It is usual to ignore non-trading days so we multiply by 252 in order to get the yearly values

In [225]:
x̄_yr = 252*x̄
σ_yr = (252*var)**0.5

### Calculating $\mu$

We calculated the empirical drift which we proved can be decomposed into two components such that

$$\mu = \bar{x} + \frac{\sigma^2}{2}$$

In [226]:
μ_yr = x̄_yr + (σ_yr**2/2)
μ_yr

0.35292502421458877

### Calculating a confidence interval on the next timestep

We can use the following to derive a confidence interval

$$ \ln S_{11} \sim \mathcal{N}\left[\ln S_{10} + \left(\mu - \frac{\sigma^2}{2} \right)\tau,  \sigma^2 \tau\right] $$

where $\tau$ is $\frac{1}{252}$ and summary statistics are anualised. Plugging in the numbers we have 

$$ \ln S_{11} \sim \mathcal{N}\left[4.926 + \left(0.353 - 0.0239 \right)\frac{1}{252},  0.0478*\frac{1}{252} \right] $$


$$ \ln S_{11} \sim \mathcal{N}\left[4.92731,  0.00018967 \right] $$

In [227]:
print(np.log(df['prices'][10]))
print((σ_yr**2)/2)
print((σ_yr**2)/252)

4.926079082538089
0.023898268031987184
0.00018966879390466018


In [228]:
avg_s = 4.926 + ((μ_yr-(σ_yr**2)/2)/252)
std_s = ((σ_yr**2)/252)**0.5

In [229]:
np.exp(avg_s + 2.57*std_s)

142.97928752678177

In [230]:
np.exp(avg_s - 2.57*std_s)

133.20796400717478